In [1]:
import numpy as np
import pandas as pd

In [2]:
from Bio import SeqIO
from Bio import Seq
from Bio.Align import MultipleSeqAlignment
from Bio.Align.AlignInfo import SummaryInfo

In [5]:
from bio_embeddings.embed import ProtTransBertBFDEmbedder


In [6]:
# Reading the files and storing in the object fold_0
fold_0 = pd.read_csv("/home/gaurav/Gaurav/Berlin/Proposals/Test/InstaDeep/data/fold_0.csv") #128
fold_1 = pd.read_csv("/home/gaurav/Gaurav/Berlin/Proposals/Test/InstaDeep/data/fold_1.csv") #129
fold_2 = pd.read_csv("/home/gaurav/Gaurav/Berlin/Proposals/Test/InstaDeep/data/fold_2.csv") #127
fold_3 = pd.read_csv("/home/gaurav/Gaurav/Berlin/Proposals/Test/InstaDeep/data/fold_3.csv") #130
fold_4 = pd.read_csv("/home/gaurav/Gaurav/Berlin/Proposals/Test/InstaDeep/data/fold_4.csv") #128
fold_test = pd.read_csv("/home/gaurav/Gaurav/Berlin/Proposals/Test/InstaDeep/data/test.csv") #36

In [7]:
fold_test["allele"] = fold_test["allele"].str[:-5] + '*' + fold_test["allele"].str[-5:]

In [8]:
frames = [fold_0, fold_1, fold_2,fold_3,fold_4,fold_test]
fold_data = pd.concat(frames)

In [9]:
embeddings_df = pd.DataFrame(np.unique(fold_data['allele']),columns=["allele"])

In [10]:
embeddings_df["allele_id"] = embeddings_df["allele"].str[-7:]
embeddings_df["hla_type"] = embeddings_df["allele"].str[:5]

In [11]:
embeddings_df

,allele,allele_id,hla_type
0,HLA-A*01:01,A*01:01,HLA-A
1,HLA-A*01:03,A*01:03,HLA-A
2,HLA-A*02:01,A*02:01,HLA-A
3,HLA-A*02:02,A*02:02,HLA-A
4,HLA-A*02:03,A*02:03,HLA-A
...,...,...,...
129,HLA-C*12:04,C*12:04,HLA-C
130,HLA-C*14:02,C*14:02,HLA-C
131,HLA-C*15:02,C*15:02,HLA-C
132,HLA-C*16:01,C*16:01,HLA-C


In [12]:
def consensus_sequence(sequences_list):
        # pad sequences so that they all have the same length
    maxlen = max(len(sequence.seq) for sequence in sequences_list)        
    for sequence in sequences_list:
        if len(sequence.seq) != maxlen:
            sequence_incomp = str(sequence.seq).ljust(maxlen, '.')
            sequence.seq = Seq.Seq(sequence_incomp)
    assert all(len(sequence.seq) == maxlen for sequence in sequences_list)

    common_alignment = MultipleSeqAlignment((sequences_list))
    summary = SummaryInfo(common_alignment)
    consensus = summary.dumb_consensus(0.6, ".")
    return consensus    

In [13]:
embeddings_df["sequence"] = "sequence"
embeddings_df["protein_embed"] = 0
embeddings_df["residue_embed"] = 0

In [14]:
HLA_type = np.unique(embeddings_df['hla_type'])
fasta_files = ["A_prot.fasta.txt","B_prot.fasta.txt","C_prot.fasta.txt"]
path_fasta = "/home/gaurav/Gaurav/Berlin/Proposals/Test/InstaDeep/data/"

In [15]:
sequences_list = []
for hla,hla_fasta in zip(HLA_type,fasta_files):
    fasta_file = path_fasta + str(hla_fasta)
    hla_list = embeddings_df[embeddings_df["hla_type"] == hla]["allele_id"]
    for var_allelle_id in hla_list:
        sequences_list = []
        for seq_record in SeqIO.parse(fasta_file, 'fasta'):
            if (seq_record.description[13:20] == var_allelle_id):
                sequences_list.append(seq_record)
        #print(var_allelle_id,)    
        cons_seq = consensus_sequence(sequences_list)
        #embeddings_df["embeddings"] = protein_embeddings(sequences_list)
        #embeddings_df["sequence"] = str(cons_seq)
        embeddings_df.loc[embeddings_df["allele_id"] == var_allelle_id ,"sequence"] = str(cons_seq)
            

In [17]:
def protein_embeddings(sequences):
    embedder = ProtTransBertBFDEmbedder()
    embeddings = embedder.embed_many([s for s in sequences])
    embeddings = list(embeddings)
    reduced_embeddings = [embedder.reduce_per_protein(e) for e in embeddings]
    return reduced_embeddings, embeddings

In [18]:
protein_emb,residue_emb = protein_embeddings(embeddings_df["sequence"])

/home/gaurav/anaconda3/envs/Zebrafish/lib/python3.7/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
Some weights of the model checkpoint at /home/gaurav/.cache/bio_embeddings/prottrans_bert_bfd/model_directory were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cl

In [80]:
embeddings_df["protein_embed"] = pd.DataFrame(protein_emb).values.tolist()
embeddings_df["residue_embed"] = np.array(residue_emb,dtype=object)

In [89]:
embeddings_df = embeddings_df.drop(columns ="proteins")

In [90]:
embeddings_df.to_csv("/home/gaurav/Gaurav/Berlin/Proposals/Test/InstaDeep/data/HLA_embeddings.csv",index=False)